In [2]:
import pandas as pd
data_path = './../../data/processed/'


In [3]:
df = pd.read_csv(data_path + 'export_bundles_qr.csv')


In [4]:
FP = df.groupby(['ID','CN ID 4', 'YEAR', 'QUARTER'])['VART'].sum().unstack([-2, -1]).sort_index(axis = 1)
FC = df.groupby(['ID','PYOD', 'YEAR', 'QUARTER'])['VART'].sum().unstack([-2, -1]).sort_index(axis = 1)
FB = df.groupby(['ID','VAT', 'YEAR', 'QUARTER'])['VART'].sum().unstack([-2, -1]).sort_index(axis = 1)

In [6]:
# # b = df_.cumsum(1)
# # c1 = b.sub(b.mask(df_).ffill(1).fillna(0)).astype(int)
# df_ = FP > 0
# b = df_.cumsum(1)
# #how long since current active period started
# c1 = b.sub(b.mask(df_).ffill(1).fillna(0)).astype(int)
# b = (~df_).cumsum(1)
# #how long since current INactive period started
# c0 = -b.sub(b.mask(~df_).ffill(1).fillna(0)).astype(int)


In [7]:
import numpy as np

def get_diff(table):
    # Interpolation, but the inerpolate method gets stuck
    ffill = (table > 0).astype(int).replace(0, np.nan).ffill(1, limit = 3)#.astype(int)
    bfill = (1 - ffill.fillna(0)).replace(0, np.nan).bfill(1, limit = 3)

    # ffilled gaps times zero when closure is definitive
    ffill = ffill.fillna(0).astype(int)
    bfill = 1 - bfill.fillna(0).astype(int)

    diff = bfill.diff(4, axis = 1) #table

    return diff

In [10]:
FB.shape

(1463609, 12)

In [24]:
diff = get_diff(FP)
diff_FP = diff.stack([-2, -1]).reset_index().rename(columns = {0: 'margin_FP'}) #long

diff = get_diff(FC)
diff_FC = diff.stack([-2, -1]).reset_index().rename(columns = {0: 'margin_FC'}) #long

In [12]:
FPC = df.groupby(['ID','PYOD', 'CN ID 4', 'YEAR', 'QUARTER'])['VART'].sum().unstack([-2, -1]).sort_index(axis = 1)

diff_4Q = FPC.fillna(0).diff(4, axis = 1) #table
diff_4Q = diff_4Q.replace(0, np.nan).stack([-2, -1]).reset_index().rename(columns = {0: 'VART_4Q_diff'}) #long

In [20]:
diff_4Q.head().merge(diff_FP, how = 'left')

,ID,PYOD,CN ID 4,YEAR,QUARTER,VART_4Q_diff,margin_FP
0,0,AD,202,1998,4,-2084.0,0.0
1,0,AD,207,1998,4,-3458.0,0.0
2,0,AD,406,1998,3,7567.0,1.0
3,0,AD,406,1999,3,-7567.0,0.0
4,0,AD,1604,1998,4,-2296.0,-1.0


In [21]:
diff_4Q.head().merge(diff_FC, how = 'left')

,ID,PYOD,CN ID 4,YEAR,QUARTER,VART_4Q_diff,margin_FC
0,0,AD,202,1998,4,-2084.0,0.0
1,0,AD,207,1998,4,-3458.0,0.0
2,0,AD,406,1998,3,7567.0,0.0
3,0,AD,406,1999,3,-7567.0,0.0
4,0,AD,1604,1998,4,-2296.0,0.0


In [26]:
diff.head(10)

YEAR       1997             1998                1999               
QUARTER       1   2   3   4    1    2    3    4    1    2    3    4
ID CN ID 4                                                         
0  202      NaN NaN NaN NaN  0.0  0.0  0.0  0.0 -1.0 -1.0 -1.0 -1.0
   207      NaN NaN NaN NaN  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0
   401      NaN NaN NaN NaN  0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0
   402      NaN NaN NaN NaN  0.0  1.0  0.0  0.0  0.0 -1.0  0.0  0.0
   404      NaN NaN NaN NaN  0.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0
   405      NaN NaN NaN NaN  1.0  1.0  0.0 -1.0 -1.0 -1.0  0.0  1.0
   406      NaN NaN NaN NaN  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0
   602      NaN NaN NaN NaN  0.0  1.0  0.0  0.0  0.0 -1.0  0.0  0.0
   701      NaN NaN NaN NaN  0.0  1.0  1.0  0.0  0.0 -1.0 -1.0  1.0
   702      NaN NaN NaN NaN  1.0  0.0  0.0  0.0 -1.0  1.0  1.0  1.0

In [27]:
FP.head(10)

YEAR           1997                            1998                     \
QUARTER           1        2   3        4         1         2        3   
ID CN ID 4                                                               
0  202      53589.0      NaN NaN   2084.0       NaN       NaN      NaN   
   207          NaN      NaN NaN  10584.0       NaN   71276.0      NaN   
   401          NaN      NaN NaN   8262.0       NaN       NaN      NaN   
   402          NaN      NaN NaN      NaN       NaN   29681.0      NaN   
   404          NaN  14236.0 NaN      NaN       NaN       NaN      NaN   
   405          NaN      NaN NaN  68289.0  114336.0  105243.0      NaN   
   406          NaN      NaN NaN      NaN    4262.0   31892.0  11915.0   
   602          NaN      NaN NaN      NaN       NaN    4744.0      NaN   
   701          NaN      NaN NaN      NaN       NaN    2332.0   4793.0   
   702          NaN      NaN NaN      NaN    2644.0       NaN      NaN   

YEAR                    1999                        
QUARTER           4        1        2   3        4  
ID CN ID 4                                          
0  202      29087.0      NaN      NaN NaN      NaN  
   207          NaN  41244.0  11964.0 NaN      NaN  
   401          NaN      NaN      NaN NaN      NaN  
   402          NaN      NaN      NaN NaN      NaN  
   404          NaN      NaN      NaN NaN      NaN  
   405          NaN      NaN      NaN NaN  46106.0  
   406       5904.0      NaN      NaN NaN   1556.0  
   602          NaN      NaN      NaN NaN      NaN  
   701          NaN      NaN      NaN NaN  11127.0  
   702          NaN      NaN   1439.0 NaN   3659.0